In [1]:
import os
os.chdir('/scratch/sagarsj42')

In [2]:
import re
import math
import datetime

import pandas as pd

In [3]:
INFO_DATA_PATH = './yt8m-info'
split = 'dev'
FILTERED_DATA_PATH = './yt8m-filtered-ids'

In [4]:
info_df = pd.read_json(os.path.join(INFO_DATA_PATH, f'{split}.jsonl'), lines=True)

print(info_df.info())

info_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30026 entries, 0 to 30025
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   vid               30026 non-null  object             
 1   title             29225 non-null  object             
 2   description       29225 non-null  object             
 3   publish_time      29225 non-null  datetime64[ns, UTC]
 4   channel_id        29225 non-null  object             
 5   channel_title     29225 non-null  object             
 6   tags              29225 non-null  object             
 7   view_count        29225 non-null  float64            
 8   like_count        29225 non-null  float64            
 9   comment_count     29225 non-null  float64            
 10  fav_count         29225 non-null  float64            
 11  duration          29225 non-null  object             
 12  definition        29225 non-null  object             
 13  l

,vid,title,description,publish_time,channel_id,channel_title,tags,view_count,like_count,comment_count,fav_count,duration,definition,licensed_content,content_rating,projection,dimension,caption
0,l2qBnK3Phks,Scorpions - WIND OF CHANGE - Nürnberg Arena 25...,The Scorpions live at the Arena in Nuremberg (...,2010-11-29 15:01:51+00:00,UCYckckJlBEc4OVaT6ds63Bw,Marco's Music Channel,"Scorpions (Musical Group),Wind Of Change,Hard ...",9640.0,23.0,0.0,0.0,5:24,hd,1.0,{},rectangular,2d,false
1,MakeMGirX6I,Stoyan Yankoulov,www.stoyankoulov.com\nSolo Tupan\n\n\nyankulov,2007-08-10 20:40:46+00:00,UChqjaIpaE8QRd1AJYLMB_fA,Stoyan Yankoulov,"World,Music,Ethno,Jazz,Folk,Drums,Percussion,E...",56723.0,232.0,16.0,0.0,2:43,sd,0.0,{},rectangular,2d,false
2,GXHQCGKux8k,Molotov - Puto - La Trastienda 27.10.10,Recital Precensal\n=),2010-10-30 15:19:03+00:00,UC1z_7LGKBe749qVfea9tOyQ,ChechuManson,MOV02982,8490.0,22.0,6.0,0.0,4:19,sd,0.0,{},rectangular,2d,false
3,cdOe_wq7_oU,Branden From Neon Trees Demos the Mike Lull 'J...,Neon Trees' Bassist Branden Campbell stopped b...,2014-08-12 20:45:48+00:00,UCI5tGbwiVHy4BsZUXcSWvwQ,Chicago Music Exchange,"cme,chicago music exchange,review,demo,Neon Tr...",43298.0,234.0,39.0,0.0,5:20,hd,1.0,{},rectangular,2d,false
4,61c9_avmN9A,"Chimpanzees feeling the Beat of the Drum "" LI...",Chimps at Chimp Haven jamming to the beat of t...,2008-08-29 08:27:04+00:00,UC9Y2dHQ4h6p9L7yVjQTdoPg,Ron Hardy,"Chimpanzee,Shreveport,Louisiana,chimp,haven,fu...",99811.0,1400.0,246.0,0.0,4:37,sd,0.0,{},rectangular,2d,false


In [5]:
duration_strs = info_df['duration'].tolist()
duration_secs = list()
for ds in duration_strs:
    if not ds or type(ds) != str:
        duration_secs.append(-1)
        continue
    
    ds_vals = ds.split(':')
    ds_sec = 0
    for i in range(1, len(ds_vals)+1):
        ds_sec += (int(ds_vals[-i]) * math.pow(60, i-1))
    duration_secs.append(ds_sec)
info_df['duration_sec'] = duration_secs

relative_date = datetime.datetime(2023, 1, 1, 0, 0, 0)
published_times = info_df['publish_time'].tolist()
age_years = list()
for pt in published_times:
    curr_date = pt.to_pydatetime().replace(tzinfo=None)
    if type(curr_date) != datetime.datetime:
        age_years.append(-1)
        continue
    curr_sec = (relative_date - curr_date).total_seconds()
    age_years.append(curr_sec/(3600*24.0*365))
info_df['age_years'] = age_years

titles = info_df['title'].tolist()
title_lens = list()
for t in titles:
    if type(t) != str:
        title_lens.append(-1)
        continue
    else:
        title_lens.append(len(t.split()))
info_df['title_len'] = title_lens

descs = info_df['description'].tolist()
desc_lens = list()
for d in descs:
    if type(d) != str:
        desc_lens.append(-1)
        continue
    d = re.sub(r'\s\s+', ' ', d)
    d = re.sub(r'\n', ' ', d)
    desc_lens.append(len(d.split()))
info_df['desc_len'] = desc_lens

tags = info_df['tags'].tolist()
n_tags = list()
for t in tags:
    if type(t) != str:
        n_tags.append(-1)
        continue
    n_tags.append(len(t.split(',')))
info_df['n_tags'] = n_tags

print(info_df.info())
info_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30026 entries, 0 to 30025
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   vid               30026 non-null  object             
 1   title             29225 non-null  object             
 2   description       29225 non-null  object             
 3   publish_time      29225 non-null  datetime64[ns, UTC]
 4   channel_id        29225 non-null  object             
 5   channel_title     29225 non-null  object             
 6   tags              29225 non-null  object             
 7   view_count        29225 non-null  float64            
 8   like_count        29225 non-null  float64            
 9   comment_count     29225 non-null  float64            
 10  fav_count         29225 non-null  float64            
 11  duration          29225 non-null  object             
 12  definition        29225 non-null  object             
 13  l

,vid,title,description,publish_time,channel_id,channel_title,tags,view_count,like_count,comment_count,...,licensed_content,content_rating,projection,dimension,caption,duration_sec,age_years,title_len,desc_len,n_tags
0,l2qBnK3Phks,Scorpions - WIND OF CHANGE - Nürnberg Arena 25...,The Scorpions live at the Arena in Nuremberg (...,2010-11-29 15:01:51+00:00,UCYckckJlBEc4OVaT6ds63Bw,Marco's Music Channel,"Scorpions (Musical Group),Wind Of Change,Hard ...",9640.0,23.0,0.0,...,1.0,{},rectangular,2d,false,324.0,12.096914,9,144,14
1,MakeMGirX6I,Stoyan Yankoulov,www.stoyankoulov.com\nSolo Tupan\n\n\nyankulov,2007-08-10 20:40:46+00:00,UChqjaIpaE8QRd1AJYLMB_fA,Stoyan Yankoulov,"World,Music,Ethno,Jazz,Folk,Drums,Percussion,E...",56723.0,232.0,16.0,...,0.0,{},rectangular,2d,false,163.0,15.403119,2,4,22
2,GXHQCGKux8k,Molotov - Puto - La Trastienda 27.10.10,Recital Precensal\n=),2010-10-30 15:19:03+00:00,UC1z_7LGKBe749qVfea9tOyQ,ChechuManson,MOV02982,8490.0,22.0,6.0,...,0.0,{},rectangular,2d,false,259.0,12.179073,7,3,1
3,cdOe_wq7_oU,Branden From Neon Trees Demos the Mike Lull 'J...,Neon Trees' Bassist Branden Campbell stopped b...,2014-08-12 20:45:48+00:00,UCI5tGbwiVHy4BsZUXcSWvwQ,Chicago Music Exchange,"cme,chicago music exchange,review,demo,Neon Tr...",43298.0,234.0,39.0,...,1.0,{},rectangular,2d,false,320.0,8.392150,13,94,19
4,61c9_avmN9A,"Chimpanzees feeling the Beat of the Drum "" LI...",Chimps at Chimp Haven jamming to the beat of t...,2008-08-29 08:27:04+00:00,UC9Y2dHQ4h6p9L7yVjQTdoPg,Ron Hardy,"Chimpanzee,Shreveport,Louisiana,chimp,haven,fu...",99811.0,1400.0,246.0,...,0.0,{},rectangular,2d,false,277.0,14.349720,10,22,13


In [6]:
info_df.describe()

,view_count,like_count,comment_count,fav_count,licensed_content,duration_sec,age_years,title_len,desc_len,n_tags
count,2.922500e+04,2.922500e+04,29225.000000,29225.0,29225.000000,30026.000000,30026.000000,30026.000000,30026.000000,30026.000000
mean,1.638110e+05,9.124709e+02,54.205304,0.0,0.111172,243.285686,10.999743,7.616532,37.400520,10.394025
std,2.783618e+06,1.350141e+04,902.458772,0.0,0.314350,99.683367,2.948848,3.741874,73.152086,10.700493
min,-1.000000e+00,-1.000000e+00,-1.000000,0.0,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,2.535000e+03,1.000000e+01,1.000000,0.0,0.000000,186.000000,9.417998,5.000000,6.000000,3.000000
50%,5.654000e+03,2.800000e+01,4.000000,0.0,0.000000,234.000000,11.074877,7.000000,15.000000,7.000000
75%,1.885400e+04,1.020000e+02,13.000000,0.0,0.000000,296.000000,12.958925,10.000000,36.000000,14.000000
max,2.939065e+08,1.263733e+06,105095.000000,0.0,1.000000,1693.000000,17.240341,23.000000,932.000000,93.000000


In [7]:
info_df = info_df[info_df['duration_sec'] >= 100]
info_df.describe()

,view_count,like_count,comment_count,fav_count,licensed_content,duration_sec,age_years,title_len,desc_len,n_tags
count,2.867800e+04,2.867800e+04,28678.000000,28678.0,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000,28678.000000
mean,1.646604e+05,9.205297e+02,54.605726,0.0,0.111375,254.765012,11.326425,7.851873,38.425413,10.712567
std,2.804387e+06,1.361279e+04,910.461902,0.0,0.314601,86.419241,2.208564,3.506149,73.807954,10.702082
min,-1.000000e+00,-1.000000e+00,-1.000000,0.0,0.000000,101.000000,7.295663,1.000000,0.000000,1.000000
25%,2.536000e+03,1.000000e+01,1.000000,0.0,0.000000,193.000000,9.554364,5.000000,7.000000,3.000000
50%,5.671500e+03,2.800000e+01,4.000000,0.0,0.000000,238.000000,11.158997,8.000000,16.000000,8.000000
75%,1.885900e+04,1.020000e+02,13.000000,0.0,0.000000,301.000000,13.025428,10.000000,37.000000,14.000000
max,2.939065e+08,1.263733e+06,105095.000000,0.0,1.000000,1693.000000,17.240341,23.000000,932.000000,93.000000


In [8]:
info_df = info_df[info_df['duration_sec'] <= 512]
info_df.describe()

,view_count,like_count,comment_count,fav_count,licensed_content,duration_sec,age_years,title_len,desc_len,n_tags
count,2.866000e+04,2.866000e+04,28660.000000,28660.0,28660.000000,28660.000000,28660.000000,28660.000000,28660.000000,28660.000000
mean,1.646359e+05,9.206800e+02,54.590126,0.0,0.111445,254.497348,11.326215,7.851396,38.422819,10.713712
std,2.805195e+06,1.361689e+04,910.713969,0.0,0.314687,85.500148,2.208638,3.505695,73.775055,10.703539
min,-1.000000e+00,-1.000000e+00,-1.000000,0.0,0.000000,101.000000,7.295663,1.000000,0.000000,1.000000
25%,2.536000e+03,1.000000e+01,1.000000,0.0,0.000000,193.000000,9.554224,5.000000,7.000000,3.000000
50%,5.673500e+03,2.800000e+01,4.000000,0.0,0.000000,238.000000,11.158742,8.000000,16.000000,8.000000
75%,1.886550e+04,1.020000e+02,13.000000,0.0,0.000000,301.000000,13.025408,10.000000,37.000000,14.000000
max,2.939065e+08,1.263733e+06,105095.000000,0.0,1.000000,511.000000,17.240341,23.000000,932.000000,93.000000


In [9]:
info_df = info_df[info_df['age_years'] >= 0]
info_df.describe()

,view_count,like_count,comment_count,fav_count,licensed_content,duration_sec,age_years,title_len,desc_len,n_tags
count,2.866000e+04,2.866000e+04,28660.000000,28660.0,28660.000000,28660.000000,28660.000000,28660.000000,28660.000000,28660.000000
mean,1.646359e+05,9.206800e+02,54.590126,0.0,0.111445,254.497348,11.326215,7.851396,38.422819,10.713712
std,2.805195e+06,1.361689e+04,910.713969,0.0,0.314687,85.500148,2.208638,3.505695,73.775055,10.703539
min,-1.000000e+00,-1.000000e+00,-1.000000,0.0,0.000000,101.000000,7.295663,1.000000,0.000000,1.000000
25%,2.536000e+03,1.000000e+01,1.000000,0.0,0.000000,193.000000,9.554224,5.000000,7.000000,3.000000
50%,5.673500e+03,2.800000e+01,4.000000,0.0,0.000000,238.000000,11.158742,8.000000,16.000000,8.000000
75%,1.886550e+04,1.020000e+02,13.000000,0.0,0.000000,301.000000,13.025408,10.000000,37.000000,14.000000
max,2.939065e+08,1.263733e+06,105095.000000,0.0,1.000000,511.000000,17.240341,23.000000,932.000000,93.000000


In [10]:
info_df = info_df[info_df['view_count'] >= 10000]
info_df.describe()

,view_count,like_count,comment_count,fav_count,licensed_content,duration_sec,age_years,title_len,desc_len,n_tags
count,1.040600e+04,1.040600e+04,10406.000000,10406.0,10406.000000,10406.000000,10406.000000,10406.000000,10406.000000,10406.000000
mean,4.467279e+05,2.491893e+03,143.293004,0.0,0.198155,256.308668,11.156015,7.947338,49.178263,12.053815
std,4.642127e+06,2.251288e+04,1507.317760,0.0,0.398629,82.756129,2.297363,3.492109,86.108047,11.477148
min,1.000200e+04,-1.000000e+00,-1.000000,0.0,0.000000,111.000000,7.295663,1.000000,0.000000,1.000000
25%,1.639600e+04,6.500000e+01,6.000000,0.0,0.000000,198.000000,9.255099,5.000000,8.000000,4.000000
50%,3.080150e+04,1.650000e+02,16.000000,0.0,0.000000,241.000000,10.915426,8.000000,19.000000,9.000000
75%,8.657325e+04,4.900000e+02,46.000000,0.0,0.000000,298.000000,12.873982,10.000000,49.000000,16.000000
max,2.939065e+08,1.263733e+06,105095.000000,0.0,1.000000,500.000000,17.240341,23.000000,886.000000,93.000000


In [11]:
info_df = info_df[info_df['like_count'] >= 100]
info_df.describe()

,view_count,like_count,comment_count,fav_count,licensed_content,duration_sec,age_years,title_len,desc_len,n_tags
count,6.674000e+03,6.674000e+03,6674.000000,6674.0,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000,6674.000000
mean,6.810847e+05,3.858512e+03,217.882979,0.0,0.253521,256.242134,10.671598,8.178753,56.896014,13.042104
std,5.782620e+06,2.801919e+04,1877.939890,0.0,0.435059,79.715112,2.164420,3.466820,93.168639,12.052567
min,1.000200e+04,1.000000e+02,-1.000000,0.0,0.000000,111.000000,7.326484,1.000000,0.000000,1.000000
25%,2.607375e+04,1.760000e+02,13.000000,0.0,0.000000,201.000000,8.893495,6.000000,9.000000,4.000000
50%,5.572500e+04,3.420000e+02,30.000000,0.0,0.000000,241.000000,10.339539,8.000000,23.500000,10.000000
75%,1.597475e+05,9.640000e+02,79.000000,0.0,1.000000,296.000000,12.113918,10.000000,59.000000,18.000000
max,2.939065e+08,1.263733e+06,105095.000000,0.0,1.000000,500.000000,16.962414,23.000000,851.000000,93.000000


In [12]:
info_df = info_df[info_df['comment_count'] >= 5]
info_df.describe()

,view_count,like_count,comment_count,fav_count,licensed_content,duration_sec,age_years,title_len,desc_len,n_tags
count,6.157000e+03,6.157000e+03,6157.000000,6157.0,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000
mean,7.196324e+05,4.048913e+03,236.060257,0.0,0.256294,256.907422,10.731592,8.222836,58.526230,13.222673
std,5.975950e+06,2.852030e+04,1954.116581,0.0,0.436621,79.823363,2.181622,3.461012,94.989792,12.092747
min,1.000200e+04,1.000000e+02,5.000000,0.0,0.000000,111.000000,7.326484,1.000000,0.000000,1.000000
25%,2.699300e+04,1.870000e+02,16.000000,0.0,0.000000,202.000000,8.937935,6.000000,10.000000,5.000000
50%,5.951800e+04,3.660000e+02,34.000000,0.0,0.000000,242.000000,10.411387,8.000000,24.000000,10.000000
75%,1.706540e+05,1.055000e+03,88.000000,0.0,1.000000,297.000000,12.191134,10.000000,61.000000,18.000000
max,2.939065e+08,1.263733e+06,105095.000000,0.0,1.000000,500.000000,16.962414,23.000000,851.000000,93.000000


In [13]:
info_df[['duration_sec', 'age_years', 'n_tags', 'view_count', 'like_count', 'comment_count', 
         'title_len', 'desc_len']].describe()

,duration_sec,age_years,n_tags,view_count,like_count,comment_count,title_len,desc_len
count,6157.000000,6157.000000,6157.000000,6.157000e+03,6.157000e+03,6157.000000,6157.000000,6157.000000
mean,256.907422,10.731592,13.222673,7.196324e+05,4.048913e+03,236.060257,8.222836,58.526230
std,79.823363,2.181622,12.092747,5.975950e+06,2.852030e+04,1954.116581,3.461012,94.989792
min,111.000000,7.326484,1.000000,1.000200e+04,1.000000e+02,5.000000,1.000000,0.000000
25%,202.000000,8.937935,5.000000,2.699300e+04,1.870000e+02,16.000000,6.000000,10.000000
50%,242.000000,10.411387,10.000000,5.951800e+04,3.660000e+02,34.000000,8.000000,24.000000
75%,297.000000,12.191134,18.000000,1.706540e+05,1.055000e+03,88.000000,10.000000,61.000000
max,500.000000,16.962414,93.000000,2.939065e+08,1.263733e+06,105095.000000,23.000000,851.000000


In [14]:
test_df = info_df.query('age_years > 7 and age_years <= 8')

test_df[['duration_sec', 'age_years', 'n_tags', 'view_count', 'like_count', 'comment_count', 
         'title_len', 'desc_len']].describe()

,duration_sec,age_years,n_tags,view_count,like_count,comment_count,title_len,desc_len
count,527.000000,527.000000,527.000000,5.270000e+02,527.000000,527.000000,527.000000,527.000000
mean,256.240987,7.704406,14.013283,6.510655e+05,5668.246679,240.874763,8.736243,77.963947
std,77.856410,0.176767,11.785061,3.335856e+06,28042.156889,1248.707615,3.451549,107.553314
min,111.000000,7.326484,1.000000,1.000600e+04,101.000000,5.000000,1.000000,0.000000
25%,202.500000,7.570671,5.000000,2.483250e+04,247.500000,16.000000,6.000000,14.000000
50%,242.000000,7.710554,12.000000,5.765700e+04,541.000000,33.000000,9.000000,39.000000
75%,292.000000,7.857811,20.000000,1.737815e+05,1738.000000,85.500000,11.000000,93.000000
max,486.000000,7.998405,59.000000,4.871412e+07,462205.000000,20592.000000,20.000000,851.000000


In [15]:
dev_df = info_df.query('age_years > 8 and age_years <= 8.5')

dev_df[['duration_sec', 'age_years', 'n_tags', 'view_count', 'like_count', 'comment_count', 
         'title_len', 'desc_len']].describe()

,duration_sec,age_years,n_tags,view_count,like_count,comment_count,title_len,desc_len
count,543.000000,543.000000,543.000000,5.430000e+02,543.000000,543.000000,543.000000,543.000000
mean,255.314917,8.247896,13.322284,9.246787e+05,5742.819521,245.863720,8.655617,67.968692
std,82.417186,0.139365,12.300557,5.634160e+06,26428.225197,1282.268389,3.557799,96.022450
min,121.000000,8.000989,1.000000,1.007300e+04,100.000000,5.000000,1.000000,0.000000
25%,199.000000,8.135086,4.000000,2.603400e+04,238.000000,15.000000,6.000000,12.000000
50%,237.000000,8.239331,11.000000,5.621200e+04,488.000000,34.000000,8.000000,33.000000
75%,298.000000,8.360655,19.000000,1.665040e+05,1366.500000,79.500000,11.000000,78.000000
max,498.000000,8.497436,93.000000,8.703280e+07,286756.000000,23673.000000,23.000000,687.000000


In [16]:
train_df = info_df.query('age_years > 8.5')

train_df[['duration_sec', 'age_years', 'n_tags', 'view_count', 'like_count', 'comment_count', 
         'title_len', 'desc_len']].describe()

,duration_sec,age_years,n_tags,view_count,like_count,comment_count,title_len,desc_len
count,5087.000000,5087.000000,5087.000000,5.087000e+03,5.087000e+03,5087.000000,5087.000000,5087.000000
mean,257.146452,11.310317,13.130136,7.048486e+05,3.700342e+03,234.515038,8.123452,55.504620
std,79.756698,1.952525,12.101126,6.219825e+06,2.877630e+04,2070.128675,3.443907,93.191292
min,119.000000,8.500141,1.000000,1.000200e+04,1.000000e+02,5.000000,1.000000,0.000000
25%,202.000000,9.729571,5.000000,2.746250e+04,1.790000e+02,16.000000,6.000000,9.000000
50%,242.000000,10.957731,10.000000,6.003400e+04,3.420000e+02,35.000000,8.000000,22.000000
75%,297.000000,12.659654,18.000000,1.711240e+05,9.570000e+02,88.000000,10.000000,56.000000
max,500.000000,16.962414,81.000000,2.939065e+08,1.263733e+06,105095.000000,22.000000,850.000000


In [17]:
os.makedirs(FILTERED_DATA_PATH, exist_ok=True)
with open(os.path.join(FILTERED_DATA_PATH, 'train.txt'), 'w') as f:
    f.write('\n'.join(train_df['vid'].tolist()))
with open(os.path.join(FILTERED_DATA_PATH, 'dev.txt'), 'w') as f:
    f.write('\n'.join(dev_df['vid'].tolist()))
with open(os.path.join(FILTERED_DATA_PATH, 'test.txt'), 'w') as f:
    f.write('\n'.join(test_df['vid'].tolist()))